<a href="https://colab.research.google.com/github/casangi/cngi_prototype/blob/master/docs/prototypes/mosaic_image_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mosaic Imaging Example

This notebook will demonstrate how to create a mosaic image.

This walkthrough is designed to be run in a Jupyter notebook on Google Colaboratory. To open the notebook in colab, go [here](https://colab.research.google.com/github/casangi/cngi_prototype/blob/master/docs/prototypes/mosaic_image_example.ipynb).

## Installation


In [ ]:
import os
os.system("pip install --extra-index-url https://test.pypi.org/simple/ cngi-prototype==0.0.63")
print('complete') 

complete


# Dataset

The simulated dataset consists of three fields which contain four point sources over three frequency channels. The ALMA layout ```alma.cycle6.3.cfg``` is used, which can be found [here](https://open-bitbucket.nrao.edu/projects/CASA/repos/casa-data/browse/alma/simmos). 

In [9]:
#https://drive.google.com/file/d/1TWXq3B2nS4LmVYJDKps5sbewjgCplxjK/view?usp=sharing
id = '1TWXq3B2nS4LmVYJDKps5sbewjgCplxjK'
os.system('curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=%s"' % id)
os.system('curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk \'/download/ {print $NF}\' ./cookie`&id=%s" -o vis.zarr.zip' % id)
os.system('unzip vis.zarr.zip')


#https://drive.google.com/file/d/1bck4ayVO8khI37wKMvRQ_7ioWUlVIpem/view?usp=sharing
id = '1bck4ayVO8khI37wKMvRQ_7ioWUlVIpem'
os.system('curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=%s"' % id)
os.system('curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk \'/download/ {print $NF}\' ./cookie`&id=%s" -o vis.zarr.zip' % id)
os.system('unzip vis.zarr.zip')

print('complete') 

complete


## Load Dataset

In [ ]:
import xarray as xr
from cngi.dio import read_vis

xr.set_options(display_style="html")

casa_img_dataset = xr.open_zarr('casa_alma12m_3field.img.zarr')

infile = "alma12m_3field.vis.zarr/"
vis_dataset = xr.open_zarr(infile+"0")
global_dataset = xr.open_zarr(infile + '/global')
#vis_dataset

casa_img_dataset

<xarray.Dataset>
Dimensions:          (chan: 3, d0: 1000, d1: 720, pol: 1)
Coordinates:
  * chan             (chan) float64 3.4e+11 3.74e+11 4.08e+11
    declination      (d0, d1) float64 dask.array<chunksize=(1000, 720), meta=np.ndarray>
  * pol              (pol) float64 9.0
    right_ascension  (d0, d1) float64 dask.array<chunksize=(1000, 720), meta=np.ndarray>
Dimensions without coordinates: d0, d1
Data variables:
    IMAGE            (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    IMAGE_PBCOR      (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    MASK             (d0, d1, chan, pol) bool dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    MODEL            (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    PB               (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    PSF              (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    RESIDUAL         (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    SUMWT            (chan, pol) float64 dask.array<chunksize=(1, 1), meta=np.ndarray>
    WEIGHT           (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
Attributes:
    axisunits:            ['rad', 'rad', '', 'Hz']
    commonbeam:           [0.6773654818534851, 0.6425685882568359, 57.7981452...
    commonbeam_units:     ['arcsec', 'arcsec', 'deg']
    date_observation:     2011/05/27/04
    direction_reference:  j2000
    imagetype:            Intensity
    incr:                 [-1.939254724438144e-07, 1.939254724438144e-07, 1.0...
    object_name:          field_1
    observer:             casa simulator
    pointing_center:      12
    rest_frequency:       3.4e+11 hz
    restoringbeam:        [[[0.6773654818534851, 0.6425685882568359, 57.79814...
    spectral__reference:  lsrk
    telescope:            alma
    telescope_position:   [2.22514e+06m, -5.44031e+06m, -2.48103e+06m] (itrf)
    unit:                 Jy/beam
    velocity__type:       radio

## Grid Parameters

grid_parms = {}
grid_parms['chan_mode'] = 'cube'
grid_parms['image_size'] = [1000,720]
grid_parms['cell_size'] = [0.04,0.04]
grid_parms['fft_padding'] = 1.0

## Phase Rotation

The UVW coordinates must be rotated and the visbility DATA must be phase rotated, relative to the mosaic phase center specified by ```rotation_parms['image_phase_center']```. 

[phase_rotate documentation](https://cngi-prototype.readthedocs.io/en/latest/_api/api/ngcasa.imaging.phase_rotate.html#ngcasa.imaging.phase_rotate)

In [ ]:
from ngcasa.imaging import phase_rotate

sel_parms = {}
sel_parms['uvw_out'] = 'UVW_ROT'
sel_parms['uvw_in'] = 'UVW'
sel_parms['data_in'] = 'DATA'
sel_parms['data_out'] = 'DATA_ROT'

rotation_parms = {}
rotation_parms['image_phase_center'] = [global_dataset.FIELD_PHASE_DIR.values[1,:,0][0], global_dataset.FIELD_PHASE_DIR.values[1,:,0][1]] #field, :, ddi
rotation_parms['common_tangent_reprojection'] = True
rotation_parms['ddi'] = 0

storage_parms = {}
storage_parms['to_disk'] =  True
storage_parms['outfile'] =  infile + '/0'
storage_parms['append'] = True

vis_dataset = phase_rotate(vis_dataset, global_dataset, rotation_parms, sel_parms, storage_parms)


######################### Start phase_rotate #########################
Setting default single_precision  to  True
Setting default graph_name  to  phase_rotate
Setting default compressor  to  Blosc(cname='zstd', clevel=2, shuffle=NOSHUFFLE, blocksize=0)
Setting default chunks_on_disk  to  {}
Setting default chunks_return  to  {}
Atempting to add  , UVW_ROT, DATA_ROT  to  alma12m_3field.vis.zarr//0
Time to append and execute graph  phase_rotate 0.26558804512023926
##################### Finished appending  phase_rotate  #####################


## Make Imaging Weights

[make_imaging_weight documentation](https://cngi-prototype.readthedocs.io/en/latest/_api/api/ngcasa.imaging.make_imaging_weight.html)

In [ ]:
from ngcasa.imaging import make_imaging_weight

imaging_weights_parms = {}
imaging_weights_parms['weighting'] = 'natural'

sel_parms = {}
sel_parms['uvw'] = 'UVW_ROT'
sel_parms['data'] = 'DATA_ROT'

storage_parms = {}
storage_parms['to_disk'] = True
storage_parms['append'] =  True
storage_parms['outfile'] =  infile + '/0'
vis_dataset = make_imaging_weight(vis_dataset, imaging_weights_parms, grid_parms, sel_parms, storage_parms)

######################### Start make_imaging_weights #########################
Setting default imaging_weight  to  IMAGING_WEIGHT
Setting default graph_name  to  make_imaging_weights
Setting default compressor  to  Blosc(cname='zstd', clevel=2, shuffle=NOSHUFFLE, blocksize=0)
Setting default chunks_on_disk  to  {}
Setting default chunks_return  to  {}
WEIGHT data variable found, will use WEIGHT to calculate  IMAGING_WEIGHT
Atempting to add  , IMAGING_WEIGHT  to  alma12m_3field.vis.zarr//0
Time to append and execute graph  make_imaging_weights 0.1306769847869873
##################### Finished appending  make_imaging_weights  #####################


## Make Gridding Convolution Functions

[make_gridding_convolution_function](https://cngi-prototype.readthedocs.io/en/latest/_api/api/ngcasa.imaging.make_gridding_convolution_function.html#ngcasa.imaging.make_gridding_convolution_function)



In [ ]:
from ngcasa.imaging import make_gridding_convolution_function
import numpy as np

gcf_parms = {}
gcf_parms['function'] = 'alma_airy'
gcf_parms['list_dish_diameters'] = np.array([10.7])
gcf_parms['list_blockage_diameters'] = np.array([0.75])

unique_ant_indx = global_dataset.ANT_DISH_DIAMETER.values
unique_ant_indx[unique_ant_indx == 12.0] = 0
#unique_ant_indx[unique_ant_indx == 7.0] = 1

gcf_parms['unique_ant_indx'] = unique_ant_indx.astype(int)
#phase_center  = SkyCoord(ra='12h01m52.430856s', dec='-18d51m49.94369s', frame='fk5')
gcf_parms['image_phase_center'] = [global_dataset.FIELD_PHASE_DIR.values[1,:,0][0], global_dataset.FIELD_PHASE_DIR.values[1,:,0][1]] #np.array([phase_center.ra.rad,phase_center.dec.rad])

storage_parms = {}
storage_parms['to_disk'] =  True

gcf_dataset = make_gridding_convolution_function(vis_dataset, global_dataset, gcf_parms, grid_parms, storage_parms)

gcf_dataset

######################### Start make_gridding_convolution_function #########################
Setting default chan_tolerance_factor  to  0.005
Setting default oversampling  to  [10, 10]
Setting default max_support  to  [15, 15]
Setting default support_cut_level  to  0.025
Setting default a_chan_num_chunk  to  3
Setting default image_center  to  [500 360]
Setting default graph_name  to  make_gcf
Setting default outfile  to  dataset.gcf.zarr
Setting default append  to  False
Setting default compressor  to  Blosc(cname='zstd', clevel=2, shuffle=NOSHUFFLE, blocksize=0)
Setting default chunks_on_disk  to  {}
Setting default chunks_return  to  {}
Saving dataset to  dataset.gcf.zarr
Time to store and execute graph  make_gcf 1.9901421070098877
##################### Created new dataset with make_gcf #####################


<xarray.Dataset>
Dimensions:             (baseline: 903, chan: 3, conv_baseline: 1, conv_chan: 3, conv_pol: 1, field: 3, l: 1000, m: 720, pol: 1, u: 160, v: 160, xy: 2)
Coordinates:
  * field               (field) int64 0 1 2
  * l                   (l) int64 0 1 2 3 4 5 6 ... 993 994 995 996 997 998 999
  * m                   (m) int64 0 1 2 3 4 5 6 ... 713 714 715 716 717 718 719
  * u                   (u) int64 0 1 2 3 4 5 6 ... 153 154 155 156 157 158 159
  * v                   (v) int64 0 1 2 3 4 5 6 ... 153 154 155 156 157 158 159
  * xy                  (xy) int64 0 1
Dimensions without coordinates: baseline, chan, conv_baseline, conv_chan, conv_pol, pol
Data variables:
    CF_BASELINE_MAP     (baseline) int64 dask.array<chunksize=(301,), meta=np.ndarray>
    CF_CHAN_MAP         (chan) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    CF_POL_MAP          (pol) int64 dask.array<chunksize=(1,), meta=np.ndarray>
    CONV_KERNEL         (conv_baseline, conv_chan, conv_pol, u, v) float64 dask.array<chunksize=(1, 1, 1, 160, 160), meta=np.ndarray>
    PHASE_GRADIENT      (field, u, v) complex128 dask.array<chunksize=(1, 160, 160), meta=np.ndarray>
    PS_CORR_IMAGE       (l, m) float64 dask.array<chunksize=(1000, 720), meta=np.ndarray>
    SUPPORT             (conv_baseline, conv_chan, conv_pol, xy) int64 dask.array<chunksize=(1, 1, 1, 2), meta=np.ndarray>
    WEIGHT_CONV_KERNEL  (conv_baseline, conv_chan, conv_pol, u, v) float64 dask.array<chunksize=(1, 1, 1, 160, 160), meta=np.ndarray>
Attributes:
    cell_uv:        [-429.71834634811734, 596.8310365946074]
    make_gcf_time:  1.9901421070098877
    oversampling:   [10, 10]

## Make Mosaic Primary Beam

[make_mosaic_pb](https://cngi-prototype.readthedocs.io/en/latest/_api/api/ngcasa.imaging.make_mosaic_pb.html#ngcasa.imaging.make_mosaic_pb)

In [ ]:
from ngcasa.imaging import make_mosaic_pb

sel_parms = {}
sel_parms['pb'] = 'MOSAIC_PB'
sel_parms['weight'] = 'WEIGHT_PB'
sel_parms['imaging_weight'] = 'IMAGING_WEIGHT'
sel_parms['uvw'] = 'UVW_ROT'

storage_parms = {}
storage_parms['to_disk'] =  True
storage_parms['outfile'] = 'mosaic_alma12m_3field.img.zarr'

img_dataset= xr.Dataset()
img_dataset = make_mosaic_pb(vis_dataset,gcf_dataset,img_dataset,sel_parms,grid_parms,storage_parms)

img_dataset

######################### Start make_mosaic_pb #########################
Setting default weight_pb  to  WEIGHT_PB
Setting default weight_pb_sum_weight  to  WEIGHT_PB_SUM_WEIGHT
Setting default image_center  to  [500 360]
Setting default graph_name  to  make_mosaic_pb
Setting default append  to  False
Setting default compressor  to  Blosc(cname='zstd', clevel=2, shuffle=NOSHUFFLE, blocksize=0)
Setting default chunks_on_disk  to  {}
Setting default chunks_return  to  {}
Saving dataset to  mosaic_alma12m_3field.img.zarr
Time to store and execute graph  make_mosaic_pb 1.3715908527374268
##################### Created new dataset with make_mosaic_pb #####################


<xarray.Dataset>
Dimensions:               (chan: 3, d0: 1000, d1: 720, pol: 1)
Coordinates:
  * chan                  (chan) float64 3.4e+11 3.74e+11 4.08e+11
    chan_width            (chan) float64 dask.array<chunksize=(1,), meta=np.ndarray>
  * d0                    (d0) int64 0 1 2 3 4 5 6 ... 994 995 996 997 998 999
  * d1                    (d1) int64 0 1 2 3 4 5 6 ... 714 715 716 717 718 719
  * pol                   (pol) int64 0
Data variables:
    MOSAIC_PB             (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    WEIGHT_PB             (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    WEIGHT_PB_SUM_WEIGHT  (chan, pol) float64 dask.array<chunksize=(1, 1), meta=np.ndarray>
Attributes:
    make_mosaic_pb_time:  1.3715908527374268

## Make Image

[make_image_with_gcf](https://cngi-prototype.readthedocs.io/en/latest/_api/api/ngcasa.imaging.make_image_with_gcf.html#ngcasa.imaging.make_image_with_gcf)

In [ ]:
from ngcasa.imaging import make_image_with_gcf

select_parms = {}
select_parms['uvw'] = 'UVW_ROT'
select_parms['data'] = 'DATA_ROT'
select_parms['imaging_weight'] = 'IMAGING_WEIGHT'
select_parms['sum_weight'] = 'IMGAGING_SUM_WEIGHT'
select_parms['image'] = 'JOINT_MOSAIC_IMAGE'
select_parms['pb'] = 'MOSAIC_PB'
select_parms['weight_pb'] = 'WEIGHT_PB'

storage_parms = {}
storage_parms['to_disk'] =  True
storage_parms['append'] = True
storage_parms['outfile'] = 'mosaic_alma12m_3field.img.zarr'

norm_parms = {}
#norm_parms['norm_type'] = 'none'
norm_parms['norm_type'] = 'flat_sky'
#norm_parms['norm_type'] = 'flat_noise'

img_dataset = make_image_with_gcf(vis_dataset,gcf_dataset, img_dataset, grid_parms, norm_parms, select_parms, storage_parms)

img_dataset

######################### Start make_image_with_gcf #########################
Setting default image_center  to  [500 360]
Setting default single_precision  to  True
Setting default graph_name  to  make_image
Setting default compressor  to  Blosc(cname='zstd', clevel=2, shuffle=NOSHUFFLE, blocksize=0)
Setting default chunks_on_disk  to  {}
Setting default chunks_return  to  {}
grid sizes 1200 864
Atempting to add  , JOINT_MOSAIC_IMAGE, IMGAGING_SUM_WEIGHT  to  mosaic_alma12m_3field.img.zarr
Time to append and execute graph  make_image 1.187060832977295
##################### Finished appending  make_image  #####################


<xarray.Dataset>
Dimensions:               (chan: 3, d0: 1000, d1: 720, pol: 1)
Coordinates:
  * chan                  (chan) float64 3.4e+11 3.74e+11 4.08e+11
    chan_width            (chan) float64 dask.array<chunksize=(1,), meta=np.ndarray>
  * d0                    (d0) int64 0 1 2 3 4 5 6 ... 994 995 996 997 998 999
  * d1                    (d1) int64 0 1 2 3 4 5 6 ... 714 715 716 717 718 719
  * pol                   (pol) int64 0
Data variables:
    IMGAGING_SUM_WEIGHT   (chan, pol) float64 dask.array<chunksize=(1, 1), meta=np.ndarray>
    JOINT_MOSAIC_IMAGE    (d0, d1, chan, pol) float32 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    MOSAIC_PB             (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    WEIGHT_PB             (d0, d1, chan, pol) float64 dask.array<chunksize=(1000, 720, 1, 1), meta=np.ndarray>
    WEIGHT_PB_SUM_WEIGHT  (chan, pol) float64 dask.array<chunksize=(1, 1), meta=np.ndarray>
Attributes:
    make_image_time:      1.187060832977295
    make_mosaic_pb_time:  1.3715908527374268

## Compare CASA and ngCASA Primary Beams

In [ ]:
import matplotlib.pylab as plt
import numpy as np
from ipywidgets import interactive
import scipy
from scipy.signal import decimate

casa_img_dataset = xr.open_zarr('casa_alma12m_3field.img.zarr')
pb_limit = 0.2


def comparison_plots(chan):
    print('Frequency',img_dataset.chan[chan].values/10**9, 'GHz')
    mosaic_pb = img_dataset.MOSAIC_PB[:,:,chan,0]
    mosaic_pb = mosaic_pb.where(mosaic_pb > pb_limit,other=np.nan)
    
    casa_mosaic_pb = casa_img_dataset.PB[:,:,chan,0]
    casa_mosaic_pb = casa_mosaic_pb.where(casa_mosaic_pb > pb_limit,other=np.nan)
    
    fig0, ax0 = plt.subplots(1, 2, sharey=True)
    im0 = ax0[0].imshow(mosaic_pb)
    im1 = ax0[1].imshow(casa_mosaic_pb)
    ax0[0].title.set_text('ngCASA Mosaic PB')
    ax0[1].title.set_text('CASA Mosaic PB')
    fig0.colorbar(im0, ax=ax0[0], fraction=0.046, pad=0.04)
    fig0.colorbar(im1, ax=ax0[1], fraction=0.046, pad=0.04)
    
    plt.figure()
    plt.plot(mosaic_pb[:,360],label='ngCASA PB')
    plt.plot(casa_mosaic_pb[:,360],'*',label='CASA PB',markersize=0.5)
    plt.legend()
    plt.title('PB cross section')
    
    plt.figure()
    plt.imshow(mosaic_pb - casa_mosaic_pb)
    plt.colorbar()
    plt.title('Difference Mosaic PB')
    
    plt.show()



interactive_plot = interactive(comparison_plots, chan=(0, 2))
output = interactive_plot.children[-1]
output.layout.height = '850px'
interactive_plot


interactive(children=(IntSlider(value=1, description='chan', max=2), Output(layout=Layout(height='850px'))), _…

## Compare CASA and ngCASA Sky Images

In [ ]:
import matplotlib.pylab as plt
import numpy as np
from ipywidgets import interactive
import scipy
from scipy.signal import decimate

casa_img_dataset = xr.open_zarr('casa_alma12m_3field.img.zarr')
pb_limit = 0.2



def comparison_plots(chan):
    print('Frequency',img_dataset.chan[chan].values/10**9, 'GHz')
    mosaic_pb = img_dataset.MOSAIC_PB[:,:,chan,0]
    casa_mosaic_pb = casa_img_dataset.PB[:,:,chan,0]
    
    mosaic_img = img_dataset.JOINT_MOSAIC_IMAGE[:,:,chan,0]
    mosaic_img = mosaic_img.where(mosaic_pb > pb_limit,other=np.nan)
    
    casa_mosaic_img = casa_img_dataset.IMAGE_PBCOR[:,:,chan,0]
    casa_mosaic_img = casa_mosaic_img.where(casa_mosaic_pb > pb_limit,other=np.nan)
    
    ngcasa_sources = np.array([img_dataset.JOINT_MOSAIC_IMAGE[313,360,chan,0].data.compute(), img_dataset.JOINT_MOSAIC_IMAGE[500,360,chan,0].data.compute(), img_dataset.JOINT_MOSAIC_IMAGE[687,360,chan,0].data.compute(),img_dataset.JOINT_MOSAIC_IMAGE[561,534,chan,0].data.compute()])
    casa_sources = np.array([casa_img_dataset.IMAGE_PBCOR[313,360,chan,0].data.compute(), casa_img_dataset.IMAGE_PBCOR[500,360,chan,0].data.compute(), casa_img_dataset.IMAGE_PBCOR[687,360,chan,0].data.compute(),casa_img_dataset.IMAGE_PBCOR[561,534,chan,0].data.compute()])

    sim_sources = np.array([2.0,1.76,1.5,2.0])
    
    print('################## Flux of Point Sources ##################')
    print('Sim   ','ngCASA  ', 'CASA')
    print('2.00  ','{0:.4f}  '.format(ngcasa_sources[0]),'{0:.4f}'.format(casa_sources[0]))
    print('1.76  ','{0:.4f}  '.format(ngcasa_sources[1]),'{0:.4f}'.format(casa_sources[1]))
    print('1.50  ','{0:.4f}  '.format(ngcasa_sources[2]),'{0:.4f}'.format(casa_sources[2]))
    print('2.00  ','{0:.4f}  '.format(ngcasa_sources[3]),'{0:.4f}'.format(casa_sources[3]))
    
    ngcasa_sources = 100*(sim_sources - ngcasa_sources)/sim_sources
    casa_sources = 100*(sim_sources - casa_sources)/sim_sources
    print('###########################################################')
    
    print('############ Percentage Difference Flux to Sim ############')
    print('ngCASA  ', 'CASA')
    print('{0:.4f}  '.format(ngcasa_sources[0]),'{0:.4f}'.format(casa_sources[0]))
    print('{0:.4f}  '.format(ngcasa_sources[1]),'{0:.4f}'.format(casa_sources[1]))
    print('{0:.4f}  '.format(ngcasa_sources[2]),'{0:.4f}'.format(casa_sources[2]))
    print('{0:.4f}  '.format(ngcasa_sources[3]),'{0:.4f}'.format(casa_sources[3]))
    print('###########################################################')
    
    
    fig0, ax0 = plt.subplots(1, 2, sharey=True)
    im0 = ax0[0].imshow(mosaic_img)
    im1 = ax0[1].imshow(casa_mosaic_img)
    ax0[0].title.set_text('ngCASA Mosaic PB')
    ax0[1].title.set_text('CASA Mosaic PB')
    fig0.colorbar(im0, ax=ax0[0], fraction=0.046, pad=0.04)
    fig0.colorbar(im1, ax=ax0[1], fraction=0.046, pad=0.04)
    
    
    plt.figure()
    plt.imshow(100*(mosaic_img - casa_mosaic_img)/2)
    plt.colorbar()
    plt.title('Difference Mosaic Image')
    
    plt.show()

interactive_plot = interactive(comparison_plots, chan=(0, 2))
output = interactive_plot.children[-1]
output.layout.height = '850px'
interactive_plot


interactive(children=(IntSlider(value=1, description='chan', max=2), Output(layout=Layout(height='850px'))), _…